# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --mem-fraction-static 0.8"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-04-22 04:40:34] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=32570, mem_fraction_static=0.8, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=1068061690, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None

[2025-04-22 04:40:44 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-22 04:40:44 TP0] Init torch distributed begin.


[2025-04-22 04:40:45 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-22 04:40:45 TP0] Load weight begin. avail mem=58.38 GB


[2025-04-22 04:40:45 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-22 04:40:45 TP0] Using model weights format ['*.safetensors']


...

[2025-04-22 04:40:46 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-04-22 04:40:47] INFO:     Started server process [1285912]
[2025-04-22 04:40:47] INFO:     Waiting for application startup.
[2025-04-22 04:40:47] INFO:     Application startup complete.
[2025-04-22 04:40:47] INFO:     Uvicorn running on http://0.0.0.0:32570 (Press CTRL+C to quit)


[2025-04-22 04:40:48] INFO:     127.0.0.1:35370 - "GET /v1/models HTTP/1.1" 200 OK
[2025-04-22 04:40:48] INFO:     127.0.0.1:35372 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-22 04:40:48 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 04:40:52] INFO:     127.0.0.1:35380 - "POST /generate HTTP/1.1" 200 OK
[2025-04-22 04:40:52] The server is fired up and ready to roll!


Server started on http://localhost:32570


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-22 04:40:53 TP0] Prefill batch. #new-seq: 1, #new-token: 37, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 04:40:53 TP0] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 5.92, #queue-req: 0, 
[2025-04-22 04:40:53] INFO:     127.0.0.1:60724 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-04-22 04:40:53 TP0] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 5, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 04:40:53 TP0] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, gen throughput (token/s): 111.92, #queue-req: 0, 


[2025-04-22 04:40:54 TP0] Decode batch. #running-req: 1, #token: 128, token usage: 0.01, gen throughput (token/s): 110.59, #queue-req: 0, 


[2025-04-22 04:40:54 TP0] Decode batch. #running-req: 1, #token: 168, token usage: 0.01, gen throughput (token/s): 142.17, #queue-req: 0, 
[2025-04-22 04:40:54] INFO:     127.0.0.1:60724 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-04-22 04:40:54] INFO:     127.0.0.1:60724 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-04-22 04:40:54 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0, 
Yes, I am Qwen, created by Alibaba

 Cloud. I am trained on a variety of natural language tasks and can provide[2025-04-22 04:40:54 TP0] Decode batch. #running-req: 1, #token: 60, token usage: 0.00, gen throughput (token/s): 128.34, #queue-req: 0, 
 helpful responses

 to users. Please let me know if you have any

 other questions or if there is anything else I can assist you with.

## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-04-22 04:40:55 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-22 04:40:55 TP0] Decode batch. #running-req: 1, #token: 21, token usage: 0.00, gen throughput (token/s): 134.24, #queue-req: 0, 


[2025-04-22 04:40:55 TP0] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, gen throughput (token/s): 141.67, #queue-req: 0, 
[2025-04-22 04:40:55] INFO:     127.0.0.1:60724 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-04-22 04:40:55 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-22 04:40:55] INFO:     127.0.0.1:60724 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "qwen/qwen2.5-0.5b-instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-04-22 04:40:55] INFO:     127.0.0.1:60740 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-22 04:40:55] INFO:     127.0.0.1:60740 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-22 04:40:55 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 48, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-04-22 04:40:56 TP0] Decode batch. #running-req: 2, #token: 86, token usage: 0.00, gen throughput (token/s): 108.44, #queue-req: 0, 


Batch job status: validating...trying again in 3 seconds...
[2025-04-22 04:40:58] INFO:     127.0.0.1:60740 - "GET /v1/batches/batch_2644bdb0-efb9-491d-8cb2-338ed8c384ee HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-04-22 04:40:58] INFO:     127.0.0.1:60740 - "GET /v1/files/backend_result_file-0c6ac3e6-1ece-40e5-9253-107e2b7783d0/content HTTP/1.1" 200 OK


[2025-04-22 04:40:58] INFO:     127.0.0.1:60740 - "DELETE /v1/files/backend_result_file-0c6ac3e6-1ece-40e5-9253-107e2b7783d0 HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-04-22 04:40:58] INFO:     127.0.0.1:60748 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-22 04:40:58] INFO:     127.0.0.1:60748 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-22 04:40:58 TP0] Prefill batch. #new-seq: 17, #new-token: 517, #cached-token: 51, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-22 04:40:58 TP0] Prefill batch. #new-seq: 3, #new-token: 93, #cached-token: 9, token usage: 0.03, #running-req: 17, #queue-req: 0, 


[2025-04-22 04:40:58 TP0] Decode batch. #running-req: 20, #token: 823, token usage: 0.04, gen throughput (token/s): 84.77, #queue-req: 0, 


[2025-04-22 04:40:59 TP0] Decode batch. #running-req: 20, #token: 1623, token usage: 0.08, gen throughput (token/s): 2361.41, #queue-req: 0, 


[2025-04-22 04:41:08] INFO:     127.0.0.1:50452 - "GET /v1/batches/batch_9035bc37-5704-4ce5-bdce-0593257b75c9 HTTP/1.1" 200 OK


[2025-04-22 04:41:11] INFO:     127.0.0.1:50452 - "GET /v1/batches/batch_9035bc37-5704-4ce5-bdce-0593257b75c9 HTTP/1.1" 200 OK


[2025-04-22 04:41:14] INFO:     127.0.0.1:50452 - "GET /v1/batches/batch_9035bc37-5704-4ce5-bdce-0593257b75c9 HTTP/1.1" 200 OK


[2025-04-22 04:41:17] INFO:     127.0.0.1:50452 - "GET /v1/batches/batch_9035bc37-5704-4ce5-bdce-0593257b75c9 HTTP/1.1" 200 OK


[2025-04-22 04:41:20] INFO:     127.0.0.1:50452 - "GET /v1/batches/batch_9035bc37-5704-4ce5-bdce-0593257b75c9 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "qwen/qwen2.5-0.5b-instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-04-22 04:41:23] INFO:     127.0.0.1:42896 - "POST /v1/files HTTP/1.1" 200 OK
[2025-04-22 04:41:23] INFO:     127.0.0.1:42896 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-04-22 04:41:25 TP0] Prefill batch. #new-seq: 43, #new-token: 710, #cached-token: 742, token usage: 0.03, #running-req: 0, #queue-req: 0, 
[2025-04-22 04:41:25 TP0] Prefill batch. #new-seq: 97, #new-token: 2910, #cached-token: 428, token usage: 0.06, #running-req: 43, #queue-req: 634, 


[2025-04-22 04:41:25 TP0] Prefill batch. #new-seq: 22, #new-token: 660, #cached-token: 110, token usage: 0.29, #running-req: 139, #queue-req: 4838, 


[2025-04-22 04:41:25 TP0] Decode batch. #running-req: 161, #token: 8894, token usage: 0.43, gen throughput (token/s): 158.04, #queue-req: 4838, 
[2025-04-22 04:41:25 TP0] Prefill batch. #new-seq: 2, #new-token: 60, #cached-token: 10, token usage: 0.48, #running-req: 160, #queue-req: 4836, 


[2025-04-22 04:41:26 TP0] Decode batch. #running-req: 159, #token: 15074, token usage: 0.74, gen throughput (token/s): 17663.04, #queue-req: 4836, 


...

[2025-04-22 04:41:34 TP0] Prefill batch. #new-seq: 29, #new-token: 3625, #cached-token: 87, token usage: 0.13, #running-req: 83, #queue-req: 10, 
[2025-04-22 04:41:34 TP0] Prefill batch. #new-seq: 10, #new-token: 1516, #cached-token: 44, token usage: 0.31, #running-req: 107, #queue-req: 0, 
[2025-04-22 04:41:34 TP0] Decode batch. #running-req: 117, #token: 8170, token usage: 0.40, gen throughput (token/s): 8124.37, #queue-req: 0, 


[2025-04-22 04:41:34 TP0] Decode batch. #running-req: 81, #token: 6868, token usage: 0.34, gen throughput (token/s): 11203.18, #queue-req: 0, 


[2025-04-22 04:41:35 TP0] Decode batch. #running-req: 78, #token: 9607, token usage: 0.47, gen throughput (token/s): 9147.66, #queue-req: 0, 


[2025-04-22 04:41:36] INFO:     127.0.0.1:46800 - "GET /v1/batches/batch_0887cab8-cfc3-4a22-a648-2a3694b1010f HTTP/1.1" 200 OK


[2025-04-22 04:41:36] INFO:     127.0.0.1:46800 - "DELETE /v1/files/backend_input_file-bf56462a-ecc3-47d7-b836-94aaacc61f20 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)

[2025-04-22 04:41:37] Child process unexpectedly failed with an exit code 9. pid=1286686
